In [3]:
# 室内的环境
import psychrolib
psychrolib.SetUnitSystem(psychrolib.SI)  # 使用国际单位制（SI）

def calculate_specific_enthalpy(temperature, relative_humidity):
    # 获取空气的压力（假设为标准大气压 101325 Pa）
    pressure = 101325  # Pa
    # 计算空气的湿度比（kg 水蒸气/kg 干空气）
    humidity_ratio = psychrolib.GetHumRatioFromRelHum(temperature, relative_humidity, pressure)
    # 计算空气的焓值（kJ/kg）
    enthalpy = psychrolib.GetMoistAirEnthalpy(temperature, humidity_ratio)
    return enthalpy

def calculate_humidity_ratio(temperature, relative_humidity):
    pressure = 101325  # Pa
    humidity_ratio = psychrolib.GetHumRatioFromRelHum(temperature, relative_humidity, pressure)
    return humidity_ratio

def calculate_relative_humidity(temperature, humidity_ratio):
    pressure = 101325  # Pa
    relative_humidity = psychrolib.GetRelHumFromHumRatio(temperature, humidity_ratio, pressure)
    return relative_humidity

# 环境参数设置
wet_bulb_temperature = 27
cooling_load_fact = 7
distribution_losses = 0.5

# 冷冻水供水温度设置
chilled_water_supply_temperature = 10

# 水泵频率与流量的关系
pump_frequency = 50
chilled_water_flow_rate = 39.09675552-2.59387582*pump_frequency+0.06112368*pump_frequency**2-0.00046424 * pump_frequency**3
pump_P_fact = 315 + ( 730 - 315 )* ( pump_frequency - 35 ) / 15

# 时间设置
set_time = 900
all_time_ratio = set_time / 3600  

# 输入空气的温度（摄氏度）和相对湿度（百分比）
temperature_indoor_air_now = 26
relative_humidity_indoor_air_now = 60

temperature_outdoor_air_now = 35
relative_humidity_outdoor_air_now = 50

temperature_fresh_air = chilled_water_supply_temperature + 8
relative_humidity_fresh_air = 90

temperature_return_air = chilled_water_supply_temperature + 8
relative_humidity_return_air = 90

# 计算焓值
enthalpy_indoor_air_now = calculate_specific_enthalpy(temperature_indoor_air_now, relative_humidity_indoor_air_now / 100) / 1000 # 室内当前空气焓值
enthalpy_outdoor_air_now = calculate_specific_enthalpy(temperature_outdoor_air_now, relative_humidity_outdoor_air_now / 100) / 1000 # 室外当前空气焓值
enthalpy_fresh_air = calculate_specific_enthalpy(temperature_fresh_air, relative_humidity_fresh_air / 100) / 1000   # 新风送风焓值
enthalpy_return_air = calculate_specific_enthalpy(temperature_return_air, relative_humidity_return_air / 100) / 1000  # 风盘送风焓值

print(f"室内空气的焓值为: {enthalpy_indoor_air_now:.2f} kJ/kg")
print(f"室外空气的焓值为: {enthalpy_outdoor_air_now:.2f} kJ/kg")
print(f"新风送风空气的焓值为: {enthalpy_fresh_air:.2f} kJ/kg")
print(f"回风送风空气的焓值为: {enthalpy_return_air:.2f} kJ/kg")

# 回水温度计算
# 空气容积
room_volume = 2000
fresh_air_volume_rated = 300  # 新风机额定送风量
return_air_volume_rated = 1000  # 回风机额定送风量
fresh_air_volume = fresh_air_volume_rated * all_time_ratio  # 新风机送风量
return_air_volume = return_air_volume_rated * all_time_ratio  # 回风机送风量

# 空气质量
air_density = 1.2
air_specific_heat_capacity = 1.005
fresh_air_quality = fresh_air_volume * air_density  # 新风机送风质量
return_air_quality = return_air_volume * air_density  # 回风机送风质量

# 计算湿度比
humidity_ratio_indoor_air_now = calculate_humidity_ratio(temperature_indoor_air_now, relative_humidity_indoor_air_now / 100)
humidity_ratio_fresh_air = calculate_humidity_ratio(temperature_fresh_air, relative_humidity_fresh_air / 100)
humidity_ratio_return_air = calculate_humidity_ratio(temperature_return_air, relative_humidity_return_air / 100)

# 计算混合空气的湿度比
mixed_humidity_ratio = (fresh_air_quality * humidity_ratio_fresh_air + return_air_quality * humidity_ratio_return_air) / (fresh_air_quality + return_air_quality)

# 转换为相对湿度
relative_humidity_indoor_air_change = calculate_relative_humidity(temperature_indoor_air_now, mixed_humidity_ratio) * 100
print(f"混合后的室内相对湿度: {relative_humidity_indoor_air_change:.2f} %")

# 空气焓差
# 空气焓差
if enthalpy_outdoor_air_now <= enthalpy_indoor_air_now:
    Q_fresh_air = 0
else:
    Q_fresh_air = fresh_air_quality * (enthalpy_outdoor_air_now - enthalpy_fresh_air) / 3600 # 新风达到送风条件需要降低的焓值 # 新风达到送风条件需要降低的焓值

Q_fresh_air_room = fresh_air_quality * ( enthalpy_indoor_air_now - enthalpy_fresh_air ) / 3600 # 算室内新风对室内温度降低贡献时用
Q_return_air = return_air_quality * ( enthalpy_indoor_air_now - enthalpy_return_air ) / 3600 # 回风达到送风条件需要降低的焓值

# 计算回水温度
chilled_water_return_temperature = chilled_water_supply_temperature +(Q_fresh_air + Q_return_air)/(4.19 * chilled_water_flow_rate  * all_time_ratio) + distribution_losses
chilled_water_temperature_change = ( chilled_water_return_temperature - chilled_water_supply_temperature )

print(f"新风需要的焓值为: {Q_fresh_air:.2f} kWh")
print(f"回风需要的焓值为: {Q_return_air:.2f} kWh")
print(f"冷冻水回水温度: {chilled_water_return_temperature:.2f} ℃")
print(f"冷冻水供回水温差: {chilled_water_temperature_change:.2f} ℃")

# 室内温度计算
net_air_heat_change =cooling_load_fact * all_time_ratio - ( Q_fresh_air_room + Q_return_air )
room_air_temperature_change = temperature_indoor_air_now + 3600 * net_air_heat_change / ( (room_volume + fresh_air_volume + return_air_volume ) * air_density * 1.005 )

print(f"变化后的室内温度: {room_air_temperature_change:.2f} ℃")

# 空调系统能耗计算
chiller_Pref = 3500
cooled_water_supply_temperature = wet_bulb_temperature + 3
chiller_plr = 4.19 * chilled_water_flow_rate * chilled_water_temperature_change / 20

chillercapftemp = 0.257896+0.0389016*chilled_water_supply_temperature-0.000217*chilled_water_supply_temperature**2+0.0468684*cooled_water_supply_temperature-0.000943*cooled_water_supply_temperature**2-0.000343*chilled_water_supply_temperature*cooled_water_supply_temperature
chillerEIRFTemp = 0.933884-0.058212*chilled_water_supply_temperature+0.00450036*chilled_water_supply_temperature**2+0.00243*cooled_water_supply_temperature+0.000486*cooled_water_supply_temperature**2-0.001215*chilled_water_supply_temperature*cooled_water_supply_temperature
chillerEIRFPLR = 0.222903+0.313387*chiller_plr+0.463710*chiller_plr**2
chiller_P_fact = chiller_Pref * chillercapftemp * chillerEIRFTemp * chillerEIRFPLR
cooled_water_return_temperature = cooled_water_supply_temperature + chilled_water_temperature_change + 3.6 * chiller_P_fact * all_time_ratio / ( 4.19 * chilled_water_flow_rate * 1000 )
all_system_energy_consumption = ( chiller_P_fact + pump_P_fact ) * all_time_ratio / 1000

print(f"PLR: {chiller_plr * 100:.2f} %")
print(f"空调功率: {chiller_P_fact:.2f} W")
print(f"冷却水回水温度: {cooled_water_return_temperature:.2f} ℃")
print(f"冷冻水流量: {chilled_water_flow_rate:.2f} m³/h")
print(f"水泵功率: {pump_P_fact:.2f} W")
print(f"总能耗: {all_system_energy_consumption:.2f} kWh")

室内空气的焓值为: 58.37 kJ/kg
室外空气的焓值为: 80.80 kJ/kg
新风送风空气的焓值为: 47.55 kJ/kg
回风送风空气的焓值为: 47.55 kJ/kg
混合后的室内相对湿度: 55.24 %
新风需要的焓值为: 0.83 kWh
回风需要的焓值为: 0.90 kWh
冷冻水回水温度: 10.90 ℃
冷冻水供回水温差: 0.90 ℃
变化后的室内温度: 26.74 ℃
PLR: 78.47 %
空调功率: 2701.22 W
冷却水回水温度: 31.03 ℃
冷冻水流量: 4.18 m³/h
水泵功率: 730.00 W
总能耗: 0.86 kWh
